In [31]:
import json
from tqdm import tqdm

In [19]:
with open("data/intents.json") as file:
    intents = json.load(file)["intents"]

In [15]:
from dotenv import load_dotenv


load_dotenv()

True

# Add intents to ElasticSearch Index

In [32]:
from modules.embeddings.transformer import TransformerEmbeddingModel


emb_model = TransformerEmbeddingModel()

In [33]:
embedding_text_template = ("""Equivalent questions: \"""{patterns}\"""
Equivalent answers:\"""{responses}\"""
""")


for intent in tqdm(intents):
    embedding_text = embedding_text_template.format(
        patterns='\n'.join(intent["patterns"]),
        responses='\n'.join(intent["responses"])
    )
    intent['text'] = embedding_text
    intent['vector_field'] = emb_model.get_embeddings([embedding_text])

100%|██████████| 172/172 [00:25<00:00,  6.83it/s]


In [34]:
from modules.elastic_search.elastic_search_client import ElasticSearchClient


es_client = ElasticSearchClient()

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: ProtocolError(('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))))

## Mappings and Index

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

## Add documents into index

In [ ]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

# Chat call

In [8]:
%load_ext autoreload
%autoreload 2
from modules.llms.openai.ollama import OllamaClient

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
prompt = "What's the formula for energy"

messages = [
    {"role": "user", "content": prompt}
]
temperature = 0.0

llm = OllamaClient()

llm.chat(messages, temperature=temperature)

NotFoundError: Error code: 404 - {'error': {'message': 'model "gemma2b" not found, try pulling it first', 'type': 'api_error', 'param': None, 'code': None}}

# ElasticSearch

In [7]:
from modules.embeddings.transformer import TransformerEmbeddingModel 

## Embeddings

In [6]:
emb_model = TransformerEmbeddingModel()
emb_model.get_embeddings(["This is a simple sentence", "This is a simple sentence"])

[array([-1.02742875e+00, -6.20887697e-01, -2.58747220e-01,  2.02031136e-01,
        -5.77992797e-01, -4.70529109e-01,  1.67170927e-01, -4.44530249e-01,
        -1.73787385e-01,  3.78513753e-01,  6.49563670e-01,  5.31426013e-01,
         6.03745401e-01, -5.84772646e-01, -1.14653826e-01, -2.65448600e-01,
         1.29080546e+00,  2.84286529e-01, -9.67649817e-01, -7.39865661e-01,
        -4.13311049e-02, -4.34799679e-02,  3.58396083e-01,  2.78101917e-02,
        -7.73855567e-01, -2.59201944e-01,  7.49615490e-01, -2.97319740e-01,
         4.30462629e-01,  8.31410885e-02,  2.20827069e-02, -5.49016595e-02,
        -5.76186441e-02, -1.66659027e-01,  7.20160902e-01,  3.14750113e-02,
        -1.36599600e-01,  3.17768663e-01, -5.80231667e-01, -1.02310026e+00,
         8.84112477e-01,  3.83380949e-01,  4.57735956e-01, -1.31415379e+00,
         3.39305848e-01,  1.40368985e-02, -8.27782929e-01,  8.46570015e-01,
        -1.30876338e+00, -1.60290480e-01, -4.10358816e-01,  2.59266108e-01,
         9.7

### Elastic search connection

In [ ]:
from modules.elastic_search.elastic_search_client import ElasticSearchClient


es_client = ElasticSearchClient()

## Query

In [ ]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}

In [ ]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"]